<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#反爬：代理伺服器/IP" data-toc-modified-id="反爬：代理伺服器/IP-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>反爬：代理伺服器/IP</a></span><ul class="toc-item"><li><span><a href="#作業目標" data-toc-modified-id="作業目標-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>作業目標</a></span></li></ul></li></ul></div>

# 反爬：代理伺服器/IP

* 了解「IP 黑/白名單」的反爬蟲機制
* 「IP 黑/白名單」反爬蟲的因應策略

## 作業目標

* 目前程式中的 proxy_ips 是手動輸入的，請根據 https://free-proxy-list.net/ 寫一個可自動化抓取可用 Proxy 的 proxy_ips。




In [1]:
from bs4 import BeautifulSoup
import requests
import random

In [2]:
proxy_ips = []

# 對 https://free-proxy-list.net/ 發送請求，並從表格中整理出免費的代理伺服器

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
url = 'https://free-proxy-list.net/'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

for tr in soup.find('tbody').find_all('tr')[:100]:
    # 取 Ip and port
    proxy = tr.find_all('td')[0].text + ':' + tr.find_all('td')[1].text
    proxy_ips.append(proxy)
    
print(proxy_ips)

['103.233.198.156:8081', '51.81.82.175:80', '40.79.26.139:1080', '216.58.163.118:80', '186.229.25.18:8080', '191.101.39.105:80', '203.146.255.101:80', '168.138.33.79:50000', '192.117.146.110:80', '144.202.101.83:3128', '104.207.151.250:3128', '27.147.155.70:55424', '43.245.202.15:57396', '192.109.165.239:80', '133.86.253.49:80', '104.42.216.178:80', '84.247.51.123:3128', '52.77.155.160:80', '41.79.33.170:8080', '124.107.185.189:3128', '178.128.125.16:34001', '103.156.225.18:80', '195.239.178.110:48009', '165.22.64.68:37196', '201.151.79.30:8080', '101.255.75.46:40092', '176.235.182.79:8080', '14.207.60.7:3128', '51.75.147.33:3128', '223.16.188.152:8908', '51.75.147.43:3128', '113.53.83.212:44664', '46.191.226.105:3128', '185.142.67.23:8080', '91.214.128.243:23500', '176.197.95.2:3128', '200.69.210.59:80', '217.15.195.141:59792', '91.192.2.168:53281', '103.106.114.134:8080', '37.187.19.70:80', '202.75.99.10:56773', '180.179.98.22:3128', '163.172.28.22:80', '181.129.183.19:53281', '125.2

In [5]:
# 將免費的代理伺服器，發送至 http://ip.filefab.com/index.php 找出可用的代理伺服器

available_proxies = []

for i in range(100):
    ip = proxy_ips[i]
    print('Use', ip)
    try:
        resp = requests.get('http://ip.filefab.com/index.php',
                        proxies={'http': ip, 'https': ip}, timeout=10)
        soup = BeautifulSoup(resp.text, 'html5lib')
        # .strip() 去除頭尾空白
        print(soup.find('h1', id='ipd').text.strip())
        available_proxies.append(ip)
        # 取 6 個可用 Proxy
        if len(available_proxies) > 5:
            break
    except:
        print('Fail')
    
print(available_proxies)

Use 103.233.198.156:8081
Fail
Use 51.81.82.175:80
Fail
Use 40.79.26.139:1080
Fail
Use 216.58.163.118:80
Fail
Use 186.229.25.18:8080
Fail
Use 191.101.39.105:80
Fail
Use 203.146.255.101:80
Fail
Use 168.138.33.79:50000
Fail
Use 192.117.146.110:80
Fail
Use 144.202.101.83:3128
Fail
Use 104.207.151.250:3128
Fail
Use 27.147.155.70:55424
Fail
Use 43.245.202.15:57396
Fail
Use 192.109.165.239:80
Fail
Use 133.86.253.49:80
Fail
Use 104.42.216.178:80
Fail
Use 84.247.51.123:3128
Fail
Use 52.77.155.160:80
Fail
Use 41.79.33.170:8080
Fail
Use 124.107.185.189:3128
Fail
Use 178.128.125.16:34001
Fail
Use 103.156.225.18:80
Fail
Use 195.239.178.110:48009
Fail
Use 165.22.64.68:37196
Fail
Use 201.151.79.30:8080
Fail
Use 101.255.75.46:40092
Fail
Use 176.235.182.79:8080
Fail
Use 14.207.60.7:3128
Fail
Use 51.75.147.33:3128
Fail
Use 223.16.188.152:8908
Fail
Use 51.75.147.43:3128
Fail
Use 113.53.83.212:44664
Fail
Use 46.191.226.105:3128
Fail
Use 185.142.67.23:8080
Fail
Use 91.214.128.243:23500
Fail
Use 176.197.95.

In [4]:
# 將前面找出的可用代理伺服器，再發送至 https://httpbin.org/ip 查看是否代理（偷換 IP）成功

for ip in available_proxies:
    print('Use', ip)
    try:
        resp = requests.get('https://httpbin.org/ip',
                        proxies={'http': ip, 'https': ip}, timeout=10)
        print(resp.json())
    except:
        print('Fail')